In [ ]:
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

In [ ]:
def read_mta(file_nums):
    path ='http://web.mta.info/developers/data/nyct/turnstile/turnstile_'
    allFiles = []
    for i in file_nums:
        allFiles.append(path + i + ".txt")
    df_list = [pd.read_csv(file) for file in allFiles]
    df = pd.concat(df_list)
    df.columns = df.columns.str.strip()
    return df

In [ ]:
df = read_mta(['191228', '191221', '191214', '191207','191130','191123','191116','191109','191102','191026','191019','191012','191005'])

In [ ]:
 df[df.DESC == 'RECOVR AUD']

In [ ]:
# remove duplicates
df = df[df.DESC != 'RECOVR AUD']
df = df[df.TIME != '04:01:13']
# Sanity check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(df
 .groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'TIME'])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()

In [ ]:
# data is at a turnstile level
df.head()

In [ ]:
df['DATETIME'] = pd.to_datetime(df.DATE + ' ' + df.TIME)

In [ ]:
df = df.set_index(['DATETIME'])
#df.head()

In [ ]:
# filter dataset to 11:59am to 8pm
afternoon = df.between_time('11:59:00', '20:01:00')
afternoon.head()

In [ ]:
# subtract max from min for each turnstile to get exits
afternoon_turn = (afternoon.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE'])['EXITS'].max()\
           -afternoon.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE'])['EXITS'].min()).reset_index()

In [ ]:
#explore outliers
afternoon_turn['EXITS'].describe()

In [ ]:
afternoon_turn['EXITS'].sort_values(ascending = True).tail(10)

In [ ]:
# removing values that appear to be counter resets
afternoon_turn = afternoon_turn[afternoon_turn.EXITS <= 11000]

In [ ]:
afternoon_turn.head()
#afternoon_turn.shape

In [ ]:
# add up total afternoon exits per station
afternoon_station = afternoon_turn.groupby(['STATION', 'DATE']).sum().reset_index()
afternoon_station['DATE'] = pd.to_datetime(afternoon_station['DATE'])
afternoon_station.head()

In [ ]:
# summing for whole month
afternoon_month = afternoon_station.groupby(['STATION'])['EXITS'].sum().reset_index()
afternoon_month = afternoon_month.set_index('STATION')

In [ ]:
# determine busiest stations
topsts = afternoon_month.sort_values('EXITS', ascending=False).head(15)
topsts.head()
topsts.plot(kind='barh', title='Afternoon Exits by Station', color = '#008080')

In [ ]:
#sns.histplot(afternoon_station['DATE'].tail(10), kde=True,bins=8, stat='density');

In [ ]:
# limit to Grand Central at turnstile level
grdcentral = df[df['STATION'] == 'GRD CNTRL-42 ST']

In [ ]:
grdcentral.shape

In [ ]:
grdcentral.head()

In [ ]:
turnstiles = grdcentral['SCP'].unique()

In [ ]:
masked_dfs = []
for i in turnstiles:
    mask = grdcentral[((grdcentral["C/A"] == "R236") & 
    (grdcentral["UNIT"] == "R045") & 
    (grdcentral["SCP"] ==  i) & 
    (grdcentral["STATION"] == "GRD CNTRL-42 ST"))]
            
    mask['DIFFS'] = mask['EXITS'].diff()
    masked_dfs.append(mask)

In [ ]:
exits_turn = pd.concat(masked_dfs)

In [ ]:
exits_turn = exits_turn[exits_turn['DIFFS'] >= 0]

In [ ]:
# now have turnstile exits per four period (diffs column)
#exits_turn.tail()
#exits_turn
exits_turn[(exits_turn.DATE=='10/04/2019')].DIFFS.sum()

In [ ]:
exits_turn.tail()

In [ ]:
# add exits across turnstiles by time
exits_turn = exits_turn.reset_index()
ct = exits_turn.groupby(['DATETIME'])['DIFFS'].sum().reset_index()

In [ ]:
ct.set_index(['DATETIME'], inplace=True)
ct.head()

In [ ]:
plt.figure(figsize=(12,8))


plt.xticks(rotation=200)
weekday1 = ct[(ct.index > '2019-11-05 23:00:00') & (ct.index < '2019-11-07 06:00:00')]
weekday1['DIFFS'].plot(title = 'Number of People Exiting Grand Central for One Day', color = '#008080' )